In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
img_width, img_height = 200,200
batch_size = 16
num_classe = 8

In [3]:
training_directory = "images/train/"

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(training_directory,
batch_size = batch_size,
class_mode="categorical",
target_size=(img_height,img_width))

Found 1750 images belonging to 8 classes.


In [4]:
validation_directory ="images/validation/" 

validation_datagen = ImageDataGenerator(rescale=1/255.0)

validation_generator = validation_datagen.flow_from_directory(validation_directory,
batch_size = batch_size,
class_mode="categorical",
target_size=(img_height,img_width))


Found 80 images belonging to 8 classes.


In [5]:
callbacks = EarlyStopping(monitor="val_loss",patience=1,verbose=1, mode="auto")
#autosave best model
best_model_file = "CNN_aug_best_model.h5"
best_model = ModelCheckpoint(best_model_file,monitor="val_acc",verbose=1,save_best_only=True)

In [6]:
# # Creating a Sequential model
# model= Sequential()
# model.add(Conv2D(kernel_size=(3,3), filters=32, activation='tanh', input_shape=(img_height,img_width,3,)))
# model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))
# model.add(MaxPooling2D(2,2))
# model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))
# model.add(MaxPooling2D(2,2))
# model.add(Conv2D(filters=30,kernel_size = (3,3),activation='tanh'))

# model.add(Flatten())

# model.add(Dense(32,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dense(8,activation = 'softmax'))

In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classe,activation="softmax")
])
#model.summary()

In [8]:
# model = Sequential([
#     Conv2D(16,(3,3),activation="relu",input_shape=(img_height,img_width,3)),MaxPooling2D(2,2),
#     Conv2D(32,(3,3),activation="relu"),MaxPooling2D(2,2),
#     Conv2D(64,(3,3),activation="relu"),
#     Conv2D(64,(3,3),activation="relu"),
#     MaxPooling2D(2,2),
#     Conv2D(128,(3,3),activation="relu"),
#     Conv2D(128,(3,3),activation="relu"),
#     MaxPooling2D(2,2),
#     Conv2D(256,(3,3),activation="relu"),
#     Conv2D(256,(3,3),activation="relu"),
#     Conv2D(256,(3,3),activation="relu"),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(512,activation="relu"),
#     Dense(512,activation="relu"),
#     Dense(num_classe,activation="softmax")
#     ])


In [9]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
history = model.fit_generator(train_generator,
epochs=10,
verbose= 1,
validation_data= validation_generator)

C:\Users\ahmet\AppData\Local\Temp\ipykernel_7440\3320234228.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
110/110 [==============================] - 22s 188ms/step - loss: 2.0804 - accuracy: 0.1200 - val_loss: 2.0794 - val_accuracy: 0.1250
Epoch 2/10
110/110 [==============================] - 20s 185ms/step - loss: 2.0797 - accuracy: 0.1200 - val_loss: 2.0794 - val_accuracy: 0.1250
Epoch 3/10
110/110 [==============================] - 20s 185ms/step - loss: 2.0798 - accuracy: 0.1263 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 4/10
110/110 [==============================] - 20s 185ms/step - loss: 2.0797 - accuracy: 0.1206 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 5/10
110/110 [==============================] - 21s 189ms/step - loss: 2.0798 - accuracy: 0.1120 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 6/10
110/110 [==============================] - 21s 189ms/step - loss: 2.0798 - accuracy: 0.1217 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 7/10
110/110 [==============================] - 21s 187ms/step - loss: 2.0797 - accuracy: 0.1223 - val_loss: 2.0794 - val_ac

In [11]:
model.save("hand_sign_detector_2.model", save_format="h5")